# Multi-tube design example (advanced) 

Design small conditional RNAs (scRNAs) for conditional Dicer substrate formation (Hochrein et al., J Am Chem Soc, 2013; see Figure 3). 

See the accompanying design specification (PDF files). See also the LaTeX spec file that you can edit to make your own design specs in a standardized format. 

This is a 2-step reaction. To design one scRNA system, there are 3 elementary step tubes plus 1 global crosstalk tube. 

Target test tubes: 
- Step 0: Reactants 
- Step 1: Intermediates
- Step 2: Products
- Global crosstalk 

Material: RNA    
Temperature: 23 C

In [1]:
# Import Python NUPACK module
from nupack import *

In [2]:
# Define physical model
my_model = Model(material='rna', celsius=37)

# Define sequence domains
a = Domain('N6', name='a')
c = Domain('N8', name='c')
b = Domain('N4', name='b')
w = Domain('N2', name='w')
y = Domain('N4', name='y')
x = Domain('N12', name='x')
z = Domain('N3', name='z')
s = Domain('N5', name='s')

# Define strands containing these domains
sC_out  = TargetStrand([w, x, y, s], name='sC_out')
sA      = TargetStrand([~c, ~b, ~a, ~z, ~y], name='sA')
sA_toe  = TargetStrand([~c], name='sA_toe')
sC      = TargetStrand([w, x, y, s, ~a, ~z, ~y, ~x, ~w], name='sC')
sC_loop = TargetStrand([s, ~a, ~z], name='sC_loop')
sB      = TargetStrand([x, y, z, a, b], name='sB')
sX      = TargetStrand([a, b, c], name='sX')

# Define target complexes
C      = TargetComplex([sC], 'D2 D12 D4( U5 U6 U3 )', name='C')
B      = TargetComplex([sB], 'U12 U4 U3 U6 U4', name='B')
C_loop = TargetComplex([sC_loop], 'U14', name='C_loop')
AB     = TargetComplex([sA, sB], 'U8 D4 D6 D3 D4(+ U12)', name='AB')
X      = TargetComplex([sX], 'U18', name='X')
XA     = TargetComplex([sX, sA], 'D6 D4 D8(+) U3 U4', name='XA')
C_out  = TargetComplex([sC_out], 'U23', name='C_out')
BC     = TargetComplex([sB, sC], 'D12 D4 D3 D6 (U4 + U2 U12 U4 U5) U2', name='BC')
A_toe  = TargetComplex([sA_toe], 'U8', name='A_toe')

# Define elementary step tubes
Step_0 = TargetTube(on_targets={C: 1e-08, X: 1e-08, AB: 1e-08}, 
    off_targets=SetSpec(max_size=2, include=[[sA], BC], exclude=[XA]), name='Step_0')
Step_1 = TargetTube(on_targets={XA: 1e-08, B: 1e-08}, 
    off_targets=SetSpec(max_size=2, include=[X, AB]), name='Step_1')
Step_2 = TargetTube(on_targets={BC: 1e-08}, 
    off_targets=SetSpec(max_size=2, include=[B, C]), name='Step_2')

# Define global crosstalk tube
Crosstalk = TargetTube(on_targets={
    AB: 1e-08,
    C: 1e-08,
    X: 1e-08,
    B: 1e-08,
    C_out: 1e-08,
    C_loop: 1e-08,
    A_toe: 1e-08,
}, off_targets=SetSpec(max_size=2, exclude=[XA, BC, [sX, sA_toe], [sB, sC_loop]]), name='Crosstalk')

# Define hard sequence constraints

# GC content constraints
hard = [
    Similarity(d, 'S'*d.nt(), limits=(0.45, 0.55)) 
        for d in [sC_out, sA, sC, sC_loop, sB, sX]
]

# Biological sequences 
# Note: biological sequence constraints often require a higher stop condition
tpm3 = 'GAACACTATTAGCTATTTGTAGTACTCTAAAGAGGACTGCAGAACGCATCGCAGTAGTGGTGAAAAGCCGTGCGTGCGCGTGAAACATCTGATCCTCACGTTACTTCCACTCGCTCTGCGTTTGACTTGTTGGCGGGGCGTTGGTGCCTTGGACTTTTTTTTCCTCCTTCTCTTCTTCGCGGCTCGGTCCACTACGCTGCTCGAGAGGAATCTGCTTTATTCGACCACACTACTCCTAAAGTAACACATTAAAATGGCCGGATCAAACAGCATCGATGCAGTTAAGAGAAAAATCAAAGTTTTACAACAGCAAGCAGATGAGGCAGAAGAAAGAGCCGAGATTTTGCAGAGACAGGTCGAGGAGGAGAAGCGTGCCAGGGAGCAGGCTGAGGCAGAGGTGGCTTCTCTGAACAGGCGTATCCAGCTGGTTGAGGAGGAGTTGGATCGTGCTCAGGAGAGACTGGCCACAGCCCTGCAAAAGCTGGAGGAAGCCGAGAAGGCCGCAGATGAGAGCGAGAGAGGGATGAAGGTGATTGAGAACAGGGCTCTGAAGGATGAGGAGAAGATGGAGCTGCAGGAGATCCAGCTTAAGGAGGCCAA'
desm = 'CATTTACACAGCGTACAAACCCAACAGGCCCAGTCATGAGCACGAAATATTCAGCCTCCGCCGAGTCGGCGTCCTCTTACCGCCGCACCTTTGGCTCAGGTTTGGGCTCCTCTATTTTCGCCGGCCACGGTTCCTCAGGTTCCTCTGGCTCCTCAAGACTGACCTCCAGAGTTTACGAGGTGACCAAGAGCTCCGCTTCTCCCCATTTTTCCAGCCACCGTGCGTCCGGCTCTTTCGGAGGTGGCTCGGTGGTCCGTTCCTACGCTGGCCTTGGTGAGAAGCTGGATTTCAATCTGGCTGATGCCATAAACCAGGACTTCCTCAACACGCGTACTAATGAGAAGGCCGAGCTCCAGCACCTCAATGACCGCTTCGCCAGCTACATCGAGAAGGTGCGCTTCCTCGAGCAGCAGAACTCTGCCCTGACGGTGGAGATTGAGCGTCTGCGGGGTCGCGAGCCCACCCGTATTGCAGAGCTGTACGAGGAGGAGATGAGAGAGCTGCGCGGACAGGTGGAGGCACTGACCAATCAGAGATCCCGTGTGGAGATCGAGAGGGACAACCTAGTCGATGACCTACAGAAACTAAAGCTCAGACTTC'

# Window constraints on detection target 'a-b-c' and silencing target 'w-x-y-z'
hard += [Window([a, b, c], [tpm3])]     
hard += [Window([w, x, y, z], [desm])]  

# Diversity constraint
hard += [Diversity(word=4, types=2)]

# Set a stop condition of 8% 
# Set seed for random number generation to get a reproducible result for this demo
my_options = DesignOptions(f_stop=0.08, seed=93)

# Define and run the test tube design job
des = tube_design(tubes=[Step_0, Step_1, Step_2, Crosstalk], model=my_model, hard_constraints=hard, options=my_options)
result = des.run(trials=1)[0]

# Display the result
result

Domain,Sequence
a,AUGAGG
a*,CCUCAU
b,AGAA
b*,UUCU
c,GAUGGAGC
c*,GCUCCAUC
s,AACAC
s*,GUGUU
w,CG
w*,CG
